### Database

In [1]:
! hive --database stackoverflow_ -e 'show tables'


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.135 seconds
OK
posts
users
Time taken: 0.506 seconds, Fetched: 2 row(s)


### posts

In [2]:
%%writefile describe.hql
USE stackoverflow_;
DESCRIBE posts;

Overwriting describe.hql


In [3]:
! hive -f describe.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.24 seconds
OK
id                  	int                 	                    
post_type_id        	tinyint             	                    
date                	string              	                    
owner_user_id       	int                 	                    
parent_id           	int                 	                    
score               	int                 	                    
favorite_count      	int                 	                    
tags                	array<string>       	                    
year                	string              	                    
month               	string              	                    
	 	 
# Partition Information	 	 
# col_name            	data_type           	comment             
	 	 
year                	string              	                    
month               	string           

In [4]:
%%writefile post.hql
USE stackoverflow_;

SELECT *
FROM 
    posts
WHERE
    tags IS NOT NULL AND
    year IS NOT NULL AND
    month IS NOT NULL
LIMIT 10
;

Overwriting post.hql


In [5]:
! hive -f post.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.153 seconds
OK
4004	1	2008-08-06T21:18:47.357	525	NULL	31	16	[".net","authentication","encryption","ssl"]	2008	2008-08
7237	1	2008-08-10T16:46:12.640	799	NULL	10	2	["sql-server","database"]	2008	2008-08
8472	1	2008-08-12T04:59:35.017	1	NULL	320	323	["security","language-agnostic","captcha"]	2008	2008-08
9303	1	2008-08-12T23:04:10.770	889	NULL	12	1	["c#","regex","perl"]	2008	2008-08
9355	1	2008-08-13T00:35:50.977	234	NULL	15	5	["windows","explorer","windows-shell"]	2008	2008-08
9831	1	2008-08-13T13:49:51.963	1491425	NULL	3	NULL	["javascript","drag-and-drop","mootools"]	2008	2008-08
10260	1	2008-08-13T19:18:07.667	203	NULL	6	NULL	["asp.net"]	2008	2008-08
10810	1	2008-08-14T08:38:24.760	966	NULL	3	1	["c#","winforms","authorization","adam","azman"]	2008	2008-08
12330	1	2008-08-15T15:06:31.873	NULL	NULL	6	1	["vbscript","wmi"]	2008	2008-08
12

### users

In [6]:
%%writefile describe.hql
USE stackoverflow_;
DESCRIBE users;

Overwriting describe.hql


In [7]:
! hive -f describe.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.224 seconds
OK
id                  	int                 	                    
reputation          	int                 	                    
creation_date       	string              	                    
display_name        	string              	                    
location            	string              	                    
views               	int                 	                    
up_votes            	int                 	                    
down_votes          	int                 	                    
age                 	int                 	                    
account_id          	int                 	                    
Time taken: 1.257 seconds, Fetched: 10 row(s)


In [8]:
%%writefile user.hql
USE stackoverflow_;

SELECT *
FROM 
    users
WHERE
    creation_date IS NOT NULL AND
    display_name IS NOT NULL AND
    location IS NOT NULL
LIMIT 10
;

Overwriting user.hql


In [9]:
! hive -f user.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.183 seconds
OK
37	4146	2008-08-01T12:44:00.723	Wally Lawless	Georgetown, Canada	654	256	18	35	28
169	4825	2008-08-02T22:51:03.507	Arnold Zokas	London, United Kingdom	585	1956	14	NULL	138
194	15481	2008-08-03T10:56:49.987	Adam Haile	United States	1973	217	18	33	157
338	37578	2008-08-04T18:34:44.117	Frank Krueger	United States	3941	2411	93	36	272
480	276	2008-08-06T08:40:05.797	mdy	Asia	79	49	0	28	381
756	2358	2008-08-08T15:31:50.013	Simon Gillbee	Pearland, TX	478	352	25	45	587
923	58	2008-08-10T14:25:39.540	AnotherHowie	United Kingdom	32	2	0	45	706
1097	8036	2008-08-12T11:40:09.933	Geoff	Stow, OH	527	473	30	46	838
1462	2011	2008-08-15T17:26:18.470	crono	Germany	119	73	1	35	1101
1539	2476	2008-08-16T13:31:36.823	Ryan P	United States	233	72	9	33	1158
Time taken: 2.447 seconds, Fetched: 10 row(s)


## Hive assignment. Task2
Compare most popular tags in year 2016 with tags in 2009. Tag popularity is the number of questions (post_type_id=1) with this tag. Output top 10 tags in format:
```
tag <tab> rank in 2016 <tab> rank in 2009 <tab> tag popularity in 2016 <tab> tag popularity in 2009
```

In [10]:
%%writefile query.hql

USE stackoverflow_;

DROP TABLE IF EXISTS pop_2016;
CREATE TABLE IF NOT EXISTS
    pop_2016 AS
SELECT
    year, tag, count(tag) AS popularity
FROM
    posts
LATERAL VIEW
    explode(tags) posts AS tag
WHERE
    year = 2016 AND
    post_type_id=1
GROUP BY
    year, tag
ORDER BY
    popularity DESC
;


DROP TABLE IF EXISTS pop_2009;
CREATE TABLE IF NOT EXISTS
    pop_2009 AS
SELECT
    year, tag, count(tag) AS popularity
FROM
    posts
LATERAL VIEW
    explode(tags) posts AS tag
WHERE
    year = 2009 AND
    post_type_id=1
GROUP BY
    year, tag
ORDER BY
    popularity DESC
;

Overwriting query.hql


In [11]:
%%writefile query.hql -a


DROP TABLE IF EXISTS rank_2016;
CREATE TABLE IF NOT EXISTS
    rank_2016 AS
SELECT
    tag, popularity, rank() OVER w AS rank
FROM
    pop_2016
WINDOW w AS (PARTITION BY year ORDER BY popularity DESC)
;


DROP TABLE IF EXISTS rank_2009;
CREATE TABLE IF NOT EXISTS
    rank_2009 AS
SELECT
    tag, popularity, rank() OVER w AS rank
FROM
    pop_2009
WINDOW w AS (PARTITION BY year ORDER BY popularity DESC)
;

Appending to query.hql


In [12]:
%%writefile query.hql -a

SELECT
    rank_2016.tag, rank_2016.rank, rank_2009.rank, rank_2016.popularity, rank_2009.popularity
FROM
    rank_2016
LEFT JOIN 
    rank_2009
ON
    rank_2016.tag = rank_2009.tag
ORDER BY
    rank_2016.rank
LIMIT 10;

Appending to query.hql


In [13]:
%%bash
hive -f query.hql 2> out.log
cat out.log >&2

javascript	1	5	2771	192
java	2	2	2033	243
android	3	52	1809	25
php	4	3	1673	215
python	5	11	1585	108
c#	6	1	1519	423
html	7	14	1212	84
jquery	8	8	1167	141
ios	9	186	914	7
css	10	20	801	59



Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.202 seconds
OK
Time taken: 0.115 seconds
Query ID = jovyan_20180207141616_d7c5e124-c860-458c-b5d3-7c175130c1db
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1518008069663_0047, Tracking URL = http://b5f1542b981e:8088/proxy/application_1518008069663_0047/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1518008069663_0047
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2018-02-07 14:16:34,914 Stage-1 map = 0%,  reduce = 0%
2018-02-07 14:1

### Simplify into 1 query

In [14]:
%%writefile query_2.hql

USE stackoverflow_;

DROP TABLE IF EXISTS pop_2016;
DROP TABLE IF EXISTS pop_2009;
DROP TABLE IF EXISTS rank_2016;
DROP TABLE IF EXISTS rank_2009;


SELECT
    rank_2016.tag, rank_2016.rank, rank_2009.rank, rank_2016.popularity, rank_2009.popularity
FROM (
    SELECT
        tag, popularity, rank() OVER w AS rank
    FROM (
        SELECT
            year, tag, count(tag) AS popularity
        FROM
            posts
        LATERAL VIEW
            explode(tags) posts AS tag
        WHERE
            year = 2016 AND
            post_type_id=1
        GROUP BY
            year, tag
        ORDER BY
            popularity DESC
    ) AS pop_2016
    WINDOW w AS (PARTITION BY year ORDER BY popularity DESC)
) AS rank_2016
LEFT JOIN (
    SELECT
        tag, popularity, rank() OVER w AS rank
    FROM (
        SELECT
            year, tag, count(tag) AS popularity
        FROM
            posts
        LATERAL VIEW
            explode(tags) posts AS tag
        WHERE
            year = 2009 AND
            post_type_id=1
        GROUP BY
            year, tag
        ORDER BY
            popularity DESC
    ) AS pop_2009
    WINDOW w AS (PARTITION BY year ORDER BY popularity DESC)
) AS rank_2009
ON
    rank_2016.tag = rank_2009.tag
ORDER BY
    rank_2016.rank
LIMIT 10
;

Overwriting query_2.hql


In [15]:
%%bash
hive -f query_2.hql 2> out.log
cat out.log >&2

javascript	1	5	2771	192
java	2	2	2033	243
android	3	52	1809	25
php	4	3	1673	215
python	5	11	1585	108
c#	6	1	1519	423
html	7	14	1212	84
jquery	8	8	1167	141
ios	9	186	914	7
css	10	20	801	59



Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.18 seconds
OK
Time taken: 1.758 seconds
OK
Time taken: 0.204 seconds
OK
Time taken: 0.201 seconds
OK
Time taken: 0.224 seconds
Query ID = jovyan_20180207142121_d5d4c6a4-a145-4c88-a78c-54c6e32e95cf
Total jobs = 9
Launching Job 1 out of 9
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1518008069663_0054, Tracking URL = http://b5f1542b981e:8088/proxy/application_1518008069663_0054/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1518008069663_0054
Hadoop job information for Stage-1: number of mappers: 1; number